# Examples of Pyspark ML

In [1]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('Missing').getOrCreate()

25/02/27 15:28:48 WARN Utils: Your hostname, Ardens-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.100.50 instead (on interface en0)
25/02/27 15:28:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/27 15:28:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# Read the dataset
training = spark.read.csv('name_data.csv',header=True,inferSchema=True)

In [3]:
training.show()

+-----------------+---+----------+------+
|             Name|Age|Experience|Salary|
+-----------------+---+----------+------+
|      Kaizer Tena| 32|        10|100000|
|       Lizel Cruz| 31|        12| 34000|
|         Roy Cruz| 35|        15| 58000|
|  Mabel Sarmiento| 28|        14| 65000|
| Aerold Sarmiento| 30|        15| 79000|
|  Arden Sarmiento| 29|         8| 55000|
|  Aaron Sarmiento| 34|         9| 66000|
|Abigael Sarmiento| 26|         7| 99000|
|      Kaizer Tena| 36|        20| 11000|
+-----------------+---+----------+------+



In [4]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [5]:
training.columns

['Name', 'Age', 'Experience', 'Salary']

In [6]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=["Age","Experience"],outputCol="Independent Features")

In [7]:
output=featureassembler.transform(training)

In [8]:
output.show()

+-----------------+---+----------+------+--------------------+
|             Name|Age|Experience|Salary|Independent Features|
+-----------------+---+----------+------+--------------------+
|      Kaizer Tena| 32|        10|100000|         [32.0,10.0]|
|       Lizel Cruz| 31|        12| 34000|         [31.0,12.0]|
|         Roy Cruz| 35|        15| 58000|         [35.0,15.0]|
|  Mabel Sarmiento| 28|        14| 65000|         [28.0,14.0]|
| Aerold Sarmiento| 30|        15| 79000|         [30.0,15.0]|
|  Arden Sarmiento| 29|         8| 55000|          [29.0,8.0]|
|  Aaron Sarmiento| 34|         9| 66000|          [34.0,9.0]|
|Abigael Sarmiento| 26|         7| 99000|          [26.0,7.0]|
|      Kaizer Tena| 36|        20| 11000|         [36.0,20.0]|
+-----------------+---+----------+------+--------------------+



In [9]:
output.columns

['Name', 'Age', 'Experience', 'Salary', 'Independent Features']

In [10]:
finalized_data=output.select("Independent Features","Salary")


In [11]:
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [32.0,10.0]|100000|
|         [31.0,12.0]| 34000|
|         [35.0,15.0]| 58000|
|         [28.0,14.0]| 65000|
|         [30.0,15.0]| 79000|
|          [29.0,8.0]| 55000|
|          [34.0,9.0]| 66000|
|          [26.0,7.0]| 99000|
|         [36.0,20.0]| 11000|
+--------------------+------+



In [12]:
from pyspark.ml.regression import LinearRegression
# Train test split
train_data, test_data=finalized_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol="Independent Features",labelCol="Salary")
regressor=regressor.fit(train_data)


25/02/27 15:29:00 WARN Instrumentation: [78fbaa8c] regParam is zero, which might cause numerical instability and overfitting.
25/02/27 15:29:00 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
25/02/27 15:29:00 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [13]:
# Coefficients
regressor.coefficients

DenseVector([-903.0956, -1971.0633])

In [14]:
# Intercepts
regressor.intercept

122834.45491251857

In [15]:
# Prediciton
pred_results=regressor.evaluate(test_data)

In [16]:
pred_results.predictions.show()

+--------------------+------+-----------------+
|Independent Features|Salary|       prediction|
+--------------------+------+-----------------+
|         [30.0,15.0]| 79000|66175.63930013448|
|         [31.0,12.0]| 34000| 71185.7335127859|
|         [35.0,15.0]| 58000|61660.16150740205|
|         [36.0,20.0]| 11000|50901.74966352573|
+--------------------+------+-----------------+



In [18]:
pred_results.meanAbsoluteError,pred_results.meanSquaredError

(23393.001345894798, 788197353.1787816)